# Build a Text2SQL AI Workflow with LangChain



In this project, we will design a Text2SQL workflow using LangChain, enabling users to convert natural language queries into SQL commands seamlessly. The workflow will consist of the following components:

Query Write Chain: A large language model (LLM) generates SQL queries based on the user's input question, adhering to constraints such as selecting relevant columns and limiting the number of results.

Query Execute Chain: The SQL query is executed against a pre-defined database schema using a database engine, retrieving the necessary results.

Answer Chain: The LLM formulates a user-friendly response, presenting the query results while escaping special characters for correct rendering in markdown.

By leveraging LangChain’s capabilities, we can simplify the process of interacting with databases, allowing users to obtain meaningful insights without requiring SQL expertise.

While this has agentic components like tool-calls we are defining a fixed deterministic flow where we have more control, however less flexibility.

![](https://i.imgur.com/7WrLz9I.png)



## Install OpenAI, and LangChain dependencies

In [20]:
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14

In [21]:
# !apt-get install sqlite3 -y

## Enter Open AI API Key

In [23]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [24]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================
# We use python-dotenv to load environment variables from a .env file
# This is a best practice for managing API keys and secrets
# The .env file should contain: OPENAI_API_KEY=your_api_key_here
# ============================================================================

from dotenv import load_dotenv, find_dotenv

# find_dotenv() searches for .env file in current and parent directories
# load_dotenv() loads the environment variables from the found .env file
load_dotenv(find_dotenv())

True

## Get SQL DB Script

In [ ]:
# in case of issues download from https://drive.google.com/file/d/16mZm3C7xKpPqp_86e64uzduLpM5mPUdq/view?usp=sharing and upload
# !gdown 16mZm3C7xKpPqp_86e64uzduLpM5mPUdq

## Create Comic Store Database

In [ ]:
# ============================================================================
# DATABASE SETUP - Creating the Comic Store SQLite Database
# ============================================================================
# We'll use SQLite as our database for this demo. SQLite is a lightweight,
# file-based database that's perfect for learning and prototyping.
# ============================================================================

# Step 1: Verify SQLite is installed and check version
!sqlite3 --version

3.43.2 2023-10-10 13:08:14 1b37c146ee9ebb7acd0160c0ab1fd11017a419fa8a3187386ed8cb32b709aapl (64-bit)


In [ ]:
# Step 2: Create the database by executing the SQL script
# This creates all tables and populates them with sample data
!sqlite3 ComicStore.db ".read ./comicdb_create_script.sql"

# Step 3: Verify the tables were created successfully
!sqlite3 ComicStore.db "SELECT name FROM sqlite_master WHERE type='table';"

In [ ]:
# Step 4: Preview sample data from the Comic table
# This shows the structure and content of our comics data
%%bash
sqlite3 ComicStore.db <<EOF
.headers on
.mode column
SELECT * FROM Comic LIMIT 10;
EOF

Branch
Employee
Publisher
Comic
Inventory
Customer
Sale
SaleTransactions


In [ ]:
# ============================================================================
# LANGCHAIN SQL DATABASE INTEGRATION
# ============================================================================
# LangChain provides SQLDatabase utility to interact with SQL databases
# It automatically extracts schema information and can run queries
# ============================================================================

from langchain_community.utilities import SQLDatabase

# Create a SQLDatabase instance from a SQLite connection URI
# The "sqlite:///" prefix indicates we're using SQLite
db = SQLDatabase.from_uri("sqlite:///ComicStore.db")

# Verify the connection by checking the dialect and available tables
print(f"Database Dialect: {db.dialect}")
print(f"Available Tables: {db.get_usable_table_names()}")

ComicId  Title                            PublisherId  Genre            Price  ReleaseDate
-------  -------------------------------  -----------  ---------------  -----  -----------
1        Spider-Man: Homecoming           1            Superhero        19.99  2017-07-07 
2        Batman: Year One                 2            Superhero        14.99  1987-02-01 
3        Hellboy: Seed of Destruction     3            Supernatural     24.99  1994-10-01 
4        Saga Volume 1                    4            Fantasy          12.99  2012-03-14 
5        Transformers: All Hail Megatron  5            Science Fiction  25.99  2008-09-01 
6        X-Men: Days of Future Past       1            Superhero        18.99  1981-01-01 
7        The Killing Joke                 2            Superhero        14.99  1988-03-29 
8        Sin City: The Hard Goodbye       4            Noir             22.99  1991-06-01 
9        Usagi Yojimbo Volume 1           5            Adventure        20.99  1987-09-01 

In [54]:
# ============================================================================
# EXPLORING THE DATABASE WITH LANGCHAIN'S SQLDatabase
# ============================================================================
# The SQLDatabase utility provides methods to explore and query the database:
# - db.run(): Execute SQL queries and get results
# - db.get_table_info(): Get schema information for specific tables
# - db.get_usable_table_names(): List all available tables
# ============================================================================

# Query the Comic table to see sample data
# include_columns=True returns results as list of dictionaries with column names
print("Sample Comics Data:")
print(db.run("SELECT * FROM Comic LIMIT 5;", include_columns=True))

print("\n" + "="*50 + "\n")

# Query the Employee table
print("Sample Employee Data:")
print(db.run("SELECT * FROM Employee LIMIT 5;", include_columns=True))

Sample Comics Data:
[{'ComicId': 1, 'Title': 'Spider-Man: Homecoming', 'PublisherId': 1, 'Genre': 'Superhero', 'Price': 19.99, 'ReleaseDate': '2017-07-07'}, {'ComicId': 2, 'Title': 'Batman: Year One', 'PublisherId': 2, 'Genre': 'Superhero', 'Price': 14.99, 'ReleaseDate': '1987-02-01'}, {'ComicId': 3, 'Title': 'Hellboy: Seed of Destruction', 'PublisherId': 3, 'Genre': 'Supernatural', 'Price': 24.99, 'ReleaseDate': '1994-10-01'}, {'ComicId': 4, 'Title': 'Saga Volume 1', 'PublisherId': 4, 'Genre': 'Fantasy', 'Price': 12.99, 'ReleaseDate': '2012-03-14'}, {'ComicId': 5, 'Title': 'Transformers: All Hail Megatron', 'PublisherId': 5, 'Genre': 'Science Fiction', 'Price': 25.99, 'ReleaseDate': '2008-09-01'}]


Sample Employee Data:
[{'EmployeeId': 1, 'FirstName': 'John', 'LastName': 'Doe', 'Title': 'Manager', 'BranchId': 1, 'HireDate': '2015-06-01', 'Email': 'john.doe@comicstore.com', 'Phone': '555-1234'}, {'EmployeeId': 2, 'FirstName': 'Jane', 'LastName': 'Smith', 'Title': 'Sales Associate', 'B

In [55]:
# ============================================================================
# UNDERSTANDING TABLE SCHEMA
# ============================================================================
# get_table_info() returns the CREATE TABLE statements and sample rows
# This is crucial for the LLM to understand the database structure
# when generating SQL queries
# ============================================================================

# Get detailed schema information for specific tables
print("Schema information for Comic and Sale tables:")
print(db.get_table_info(table_names=['Comic', 'Sale']))

Schema information for Comic and Sale tables:

CREATE TABLE "Comic" (
	"ComicId" INTEGER NOT NULL, 
	"Title" NVARCHAR(100) NOT NULL, 
	"PublisherId" INTEGER NOT NULL, 
	"Genre" NVARCHAR(50), 
	"Price" NUMERIC(10, 2) NOT NULL, 
	"ReleaseDate" DATETIME, 
	PRIMARY KEY ("ComicId"), 
	FOREIGN KEY("PublisherId") REFERENCES "Publisher" ("PublisherId")
)

/*
3 rows from Comic table:
ComicId	Title	PublisherId	Genre	Price	ReleaseDate
1	Spider-Man: Homecoming	1	Superhero	19.99	2017-07-07 00:00:00
2	Batman: Year One	2	Superhero	14.99	1987-02-01 00:00:00
3	Hellboy: Seed of Destruction	3	Supernatural	24.99	1994-10-01 00:00:00
*/


CREATE TABLE "Sale" (
	"SaleId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"EmployeeId" INTEGER, 
	"SaleDate" DATETIME NOT NULL, 
	"TotalAmount" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("SaleId"), 
	FOREIGN KEY("EmployeeId") REFERENCES "Employee" ("EmployeeId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Sale table:
SaleId	C

## Overview of the Comic Store Database

In this project, we will utilize a **Comic Store Database** to demonstrate the capabilities of a Text2SQL workflow. The database schema includes the following entities and relationships:

1. **Branch**: Stores details about comic store branches, including their location and contact information.

2. **Publisher**: Contains information about publishers, such as their name, country, and the year they were established.

3. **Comic**: Represents the comics, including their title, genre, price, release date, and associated publisher.

4. **Customer**: Tracks customer details, including their contact information and location.

5. **Employee**: Holds data about store employees, including their branch, title, and hire date.

6. **Inventory**: Manages the stock of comics available at different branches.

7. **Sale**: Records sales transactions, including the employee and customer involved, as well as the total amount and sale date.

8. **SaleTransactions**: Tracks individual items within a sale, including the quantity and price of each comic sold.

### Relationships:
- A **Publisher** publishes multiple **Comics**.
- A **Branch** stocks multiple **Comics** through the **Inventory** table.
- A **Customer** makes **Sales**, which are processed by **Employees**.
- Each **Sale** contains multiple items recorded in **SaleTransactions**.

This database schema is well-suited for queries related to inventory management, sales analysis, customer interactions, and employee performance in the context of a comic store business.


![](https://i.imgur.com/YzNCLpV.png)

## Build Text2SQL Components for AI Workflow

In [ ]:
# ============================================================================
# STEP 1: CRAFTING THE SQL GENERATION PROMPT
# ============================================================================
# The prompt is the most critical component of a Text2SQL system.
# It instructs the LLM on how to:
#   1. Understand the database schema (provided via {table_info})
#   2. Generate syntactically correct SQL queries
#   3. Handle edge cases (LIMIT clause, column selection, JOINs)
#   4. Format the output consistently
#
# Key Prompt Engineering Techniques Used:
#   - Role assignment: "You are a SQLite expert"
#   - Specific instructions: "Never query for all columns"
#   - Output format specification: Structured SQLQuery/SQLResult/Answer format
#   - Guardrails: "Do not include sql markers"
#
# Reference: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/sql_database/prompt.py
# ============================================================================

from langchain_core.prompts.prompt import PromptTemplate

# Suffix that gets appended to the prompt
# {table_info} - Automatically populated with table schemas by LangChain
# {input} - The user's natural language question
PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

# Main prompt template for SQLite query generation
# The {top_k} placeholder is replaced with the k parameter value
_sqlite_prompt = """You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run,
then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per SQLite.

You can order the results to return the most informative data in the database.
Never query for all columns from a table.

You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below.
Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Pay attention to use table JOINS as necessary if you are adding relevant fields from different tables.

Generate the output in the exact following format:

SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

The SQLQuery field above should have the correct SQLite query as plain text without any formatting or code blocks.
Do not include sql or similar markers.
Do not try to explain the query, just provide the query as-is, like this: SELECT ...
"""

# Create the final prompt template by combining the main prompt with suffix
SQLITE_PROMPT = PromptTemplate.from_template(_sqlite_prompt + PROMPT_SUFFIX)

print("✅ SQL Generation Prompt created successfully!")
print(f"📋 Input variables required: {SQLITE_PROMPT.input_variables}")

In [ ]:
# ============================================================================
# STEP 2: CREATE THE SQL QUERY GENERATION CHAIN
# ============================================================================
# LangChain's create_sql_query_chain() creates a chain for SQL query generation.
# It automatically:
#   - Extracts schema information from the database
#   - Populates the {table_info} placeholder in the prompt
#   - Passes the user question to the LLM
#   - Returns the generated SQL query
#
# Chain Workflow: User Question -> Prompt (with schema) -> LLM -> SQL Query
# ============================================================================

from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain

# Initialize GPT-4o with temperature=0 for deterministic SQL generation
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the query generation chain
text2sql_chain = create_sql_query_chain(
    llm=chatgpt,
    db=db,
    prompt=SQLITE_PROMPT,
    k=5  # Default LIMIT value
)

print("Text2SQL Chain created!")

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert.\nGiven an input question, first create a syntactically correct SQLite query to run,\nthen look at the results of the query and return the answer to the input question.\n\nUnless the user specifies in the question a specific number of examples to obtain,\nquery for at most {top_k} results using the LIMIT clause as per SQLite.\n\nYou can order the results to return the most informative data in the database.\nNever query for all columns from a table.\n\nYou must query only the columns that are needed to answer the question.\nWrap each column name in double quotes (") to denote them as delimited identifiers.\n\nPay attention to use only the column names you 

In [ ]:
# Optional: View the formatted prompt that will be sent to the LLM
# This shows exactly what the model sees, including table schema placeholders
# text2sql_chain.get_prompts()[0].pretty_print()

You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run,
then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 5 results using the LIMIT clause as per SQLite.

You can order the results to return the most informative data in the database.
Never query for all columns from a table.

You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below.
Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Pay attention to use table JOINS as necessary if you are adding relevant fields from differe

In [ ]:
# ============================================================================
# TEST: Generate a SQL Query from Natural Language
# ============================================================================
# Let's test the chain by asking for the top 5 most popular comics
# The chain will generate a SQL query (not execute it yet)
# ============================================================================

response = text2sql_chain.invoke({"question": "Top 5 most popular comics"})
print("Generated SQL Query:")
print(response)

SELECT "Comic"."Title", SUM("SaleTransactions"."Quantity") AS "TotalSold"
FROM "SaleTransactions"
JOIN "Comic" ON "SaleTransactions"."ComicId" = "Comic"."ComicId"
GROUP BY "Comic"."Title"
ORDER BY "TotalSold" DESC
LIMIT 5


In [ ]:
# Execute the generated SQL query against the database
# db.run() returns the query results as a string
print("Query Results:")
print(db.run(response))

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

## Create SQL Query Write & Execute Workflow Chains

In [ ]:
# ============================================================================
# STEP 3: CREATE QUERY WRITE & EXECUTE CHAINS
# ============================================================================
# Now we'll create two connected chains:
#   1. query_write_chain: Generates SQL from natural language (same as before)
#   2. query_execute_chain: Generates SQL AND executes it against the database
#
# QuerySQLDatabaseTool: A LangChain tool that executes SQL queries
# The "|" operator (pipe) chains the components together using LCEL
# ============================================================================

from langchain_community.tools import QuerySQLDatabaseTool

# Create the SQL execution tool - this runs queries against our database
execute_query_tool = QuerySQLDatabaseTool(db=db)

# Create a fresh query generation chain (with k=10 for more results)
query_write_chain = create_sql_query_chain(
    llm=chatgpt,
    db=db,
    prompt=SQLITE_PROMPT,
    k=10  # Allow up to 10 results
)

# Chain the query generation with query execution using LCEL pipe operator
# Flow: Question -> Generate SQL -> Execute SQL -> Results
query_execute_chain = query_write_chain | execute_query_tool

print("Query Write & Execute chains created!")

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

In [ ]:
# ============================================================================
# TEST: Query Write & Execute Chains
# ============================================================================
# Test the combined chain - it generates SQL and executes it in one step
# ============================================================================

# Test the combined execute chain
result = query_execute_chain.invoke({"question": "Top 5 most popular comics"})
print("Query Result:", result)

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

In [ ]:
# ============================================================================
# ADDITIONAL TEST EXAMPLES (Commented out - similar to above)
# ============================================================================
# These cells demonstrate more query examples. Uncomment to test.
# ============================================================================

# Example 1: Same query as above (redundant, kept for reference)
# query_execute_chain.invoke({"question": "Top 5 most popular comics"})

# Example 2: Customer analysis
# query_execute_chain.invoke({"question": "Top 5 customers with most comics purchased"})

# Example 3: Revenue analysis
# query_execute_chain.invoke({"question": "Top 5 customers with most money spent"})

# Example 4: Sales performance
# query_execute_chain.invoke({"question": "Top 3 salesman with highest revenue"})

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

In [ ]:
# Merged into previous cell - see above for examples
# query_execute_chain.invoke({"question": "Top 5 customers with most comics purchased"})

"[('Tony', 'Stark', 8), ('Sarah', 'Connor', 7), ('Natasha', 'Romanoff', 6), ('Clark', 'Kent', 6), ('Diana', 'Prince', 6)]"

In [ ]:
# Merged into previous cell - see above for examples
# query_execute_chain.invoke({"question": "Top 5 customers with most money spent"})

"[('Tony', 'Stark', 164.94), ('Bruce', 'Wayne', 139.94), ('Sarah', 'Connor', 124.96), ('Clark', 'Kent', 114.96), ('Natasha', 'Romanoff', 111.96000000000001)]"

In [ ]:
# Merged into previous cell - see above for examples
# query_execute_chain.invoke({"question": "Top 3 salesman with highest revenue"})

"[('John', 'Doe', 255.89), ('Alice', 'Brown', 234.94), ('Jane', 'Smith', 234.88)]"

## Create Text2SQL AI Workflow Chain

In [56]:
# ============================================================================
# STEP 4: CREATE THE COMPLETE TEXT2SQL AI WORKFLOW
# ============================================================================
# This is the final chain that combines all components into a complete workflow:
#
#   User Question
#        |
#        v
#   [Query Write Chain] --> Generates SQL query
#        |
#        v
#   [Query Execute Chain] --> Executes SQL, gets results
#        |
#        v
#   [Answer Prompt] --> Formats question + query + results
#        |
#        v
#   [LLM] --> Generates human-readable answer
#        |
#        v
#   [StrOutputParser] --> Extracts the string response
#
# Key LCEL Concepts:
#   - RunnablePassthrough.assign(): Adds new keys to the chain's state
#   - "|" operator: Pipes output from one component to the next
# ============================================================================

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Answer generation prompt - takes the SQL query and results to create a response
# Using raw string (r"...") to avoid escape sequence warnings with backslash
answer_prompt = PromptTemplate.from_template(
    r"""Given the following user question, corresponding SQL query, and SQL result,
       create a helpful answer the user question.

       When generating the final answer in markdown from the results,
       if there are special characters in the text, such as the dollar symbol,
       ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5

       Question: {question}
       SQL Query: {query}
       SQL Result: {result}
       Answer:
    """
)

# Build the complete Text2SQL chain using LCEL (LangChain Expression Language)
text2sql_chain = (
    # Step 1: Generate SQL query and add it to the state as 'query'
    RunnablePassthrough.assign(query=query_write_chain)
    |
    # Step 2: Execute the query and add results to state as 'result'
    RunnablePassthrough.assign(result=query_execute_chain)
    |
    # Step 3: Format everything into the answer prompt
    answer_prompt
    |
    # Step 4: Send to LLM for human-readable response
    chatgpt
    |
    # Step 5: Parse the output as a string
    StrOutputParser()
)

print("Complete Text2SQL AI Workflow Chain created!")

Complete Text2SQL AI Workflow Chain created!


## Test the Text2SQL AI Workflow

In [57]:
# ============================================================================
# DEMO: Test the Complete Text2SQL Workflow
# ============================================================================
# Now let's test our complete workflow with various natural language questions.
# The chain will:
#   1. Generate SQL from the question
#   2. Execute the SQL against the database
#   3. Format a human-readable response in Markdown
# ============================================================================

from IPython.display import display, Markdown

# Test 1: Simple count query
print("Question: Total number of customers")
print("-" * 50)
response = text2sql_chain.invoke({"question": "Total number of customers"})
display(Markdown(response))

Question: Total number of customers
--------------------------------------------------


The total number of customers is **20**.

In [58]:
# Test 2: Aggregation with ranking
print("Question: What are the Top 10 most popular comics")
print("-" * 50)
response = text2sql_chain.invoke({"question": "What are the Top 10 most popular comics"})
display(Markdown(response))

Question: What are the Top 10 most popular comics
--------------------------------------------------


Here are the Top 10 most popular comics based on sales:

1. **Wolverine: Old Man Logan** - Sold 3 copies
2. **V for Vendetta** - Sold 2 copies
3. **Usagi Yojimbo Volume 1** - Sold 2 copies
4. **Transformers: All Hail Megatron** - Sold 2 copies
5. **The Killing Joke** - Sold 2 copies
6. **The Boys Volume 1** - Sold 2 copies
7. **Superman: Red Son** - Sold 2 copies
8. **Punisher: Welcome Back, Frank** - Sold 2 copies
9. **Preacher Volume 1** - Sold 2 copies
10. **Ms. Marvel Volume 1** - Sold 2 copies

These comics are ranked by the total number of copies sold, with "Wolverine: Old Man Logan" leading the list.

In [59]:
# Test 3: Customer analysis with JOIN
print("Question: Top 5 customers with most comics purchased")
print("-" * 50)
response = text2sql_chain.invoke({"question": "Top 5 customers with most comics purchased"})
display(Markdown(response))

Question: Top 5 customers with most comics purchased
--------------------------------------------------


Here are the top 5 customers who have purchased the most comics:

1. **Tony Stark** - 8 comics
2. **Sarah Connor** - 7 comics
3. **Natasha Romanoff** - 6 comics
4. **Clark Kent** - 6 comics
5. **Diana Prince** - 6 comics

These customers have shown a great interest in comics, with Tony Stark leading the list with 8 purchases.

In [ ]:
# Test 4: Revenue calculation
print("Question: Which are the top 5 customers with most money spent")
print("-" * 50)
response = text2sql_chain.invoke({"question": "Which are the top 5 customers with most money spent"})

Question: Which are the top 5 customers with most money spent
--------------------------------------------------


In [62]:
# Test 5: Employee performance analysis
print("Question: Which are the top 3 salesman with highest revenue")
print("-" * 50)
response = text2sql_chain.invoke({"question": "Which are the top 3 salesman with highest revenue"})
display(Markdown(response))

Question: Which are the top 3 salesman with highest revenue
--------------------------------------------------


The top 3 salespeople with the highest revenue are:

1. **John Doe** with a total revenue of \$255.89
2. **Alice Brown** with a total revenue of \$234.94
3. **Jane Smith** with a total revenue of \$234.88

These individuals have achieved the highest sales figures, showcasing their exceptional performance in generating revenue.

                                  SUMMARY                                    


🌟 We've built a complete **Text2SQL AI Workflow** with **LangChain** that:

  1️⃣  Accepts natural language questions from users  
  2️⃣  Generates syntactically correct SQL queries using **GPT-4o**  
  3️⃣  Executes those queries against a SQLite database  
  4️⃣  Returns human-readable, beautifully formatted answers  

**Key Components Used 🛠️:**
• `PromptTemplate` – for crafting clear, precise SQL generation instructions  
• `create_sql_query_chain` – LangChain’s built-in SQL generation chain  
• `QuerySQLDatabaseTool` – for secure and efficient SQL query execution  
• `RunnablePassthrough` – to construct modular LCEL workflows  
• `StrOutputParser` – for neatly extracting and presenting string outputs  

---
This workflow offers a **deterministic, controllable** approach to Text2SQL,
unlike fully autonomous agents, empowering you with greater transparency and control.
══════════════════════════════════════════════════════════════════════════════════